<a href="https://colab.research.google.com/github/Swathi04/Assignment4-CMPE258/blob/main/weights_and_biases_of_hyperparameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup



In [1]:
%%capture
!pip install wandb --upgrade

# workaround to fetch MNIST data
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

In [2]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Define the Sweep



## Pick a `method`

The first thing we need to define is the `method`
for choosing new parameter values.

We provide the following search `methods`:
*   **`grid` Search** – Iterate over every combination of hyperparameter values.
Very effective, but can be computationally costly.
*   **`random` Search** – Select each new combination at random according to provided `distribution`s. Surprisingly effective!
*   **`bayes`ian Search** – Create a probabilistic model of metric score as a function of the hyperparameters, and choose parameters with high probability of improving the metric. Works well for small numbers of continuous parameters but scales poorly.

We'll stick with `random`.

In [3]:
sweep_config = {
    'method': 'random'
    }

For `bayes`ian Sweeps,
you also need to tell us a bit about your `metric`.
We need to know its `name`, so we can find it in the model outputs
and we need to know whether your `goal` is to `minimize` it
(e.g. if it's the squared error)
or to `maximize` it
(e.g. if it's the accuracy).

In [4]:
metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

If you're not running a `bayes`ian Sweep, you don't have to,
but it's not a bad idea to include this in your `sweep_config` anyway,
in case you change your mind later.
It's also good reproducibility practice to keep note of things like this,
in case you, or someone else,
come back to your Sweep in 6 months or 6 years
and don't know whether `val_G_batch` is supposed to be high or low.

##Name the hyper`parameters`

Once you've picked a `method` to try out new values of the hyperparameters,
you need to define what those `parameters` are.

Most of the time, this step is straightforward:
you just give the `parameter` a name
and specify a list of legal `values`
of the parameter.

For example, when we choose the `optimizer` for our network,
there's only a finite number of options.
Here we stick with the two most popular choices, `adam` and `sgd`.
Even for hyperparameters that have potentially infinite options,
it usually only makes sense to try out
a few select `values`,
as we do here with the hidden `layer_size` and `dropout`.

In [5]:
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'fc_layer_size': {
        'values': [128, 256, 512]
        },
    'dropout': {
          'values': [0.3, 0.4, 0.5]
        },
    }

sweep_config['parameters'] = parameters_dict

It's often the case that there are hyperparameters
that we don't want to vary in this Sweep,
but which we still want to set in our `sweep_config`.

In that case, we just set the `value` directly:

In [6]:
parameters_dict.update({
    'epochs': {
        'value': 1}
    })

For a `grid` search, that's all you ever need.

For a `random` search,
all the `values` of a parameter are equally likely to be chosen on a given run.

If that just won't do,
you can instead specify a named `distribution`,
plus its parameters, like the mean `mu`
and standard deviation `sigma` of a `normal` distribution.


In [7]:
import math

parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform',
        'q': 1,
        'min': math.log(32),
        'max': math.log(256),
      }
    })

When we're finished, `sweep_config` is a nested dictionary
that specifies exactly which `parameters` we're interested in trying
and what `method` we're going to use to try them.

In [8]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'batch_size': {'distribution': 'q_log_uniform',
                               'max': 5.545177444479562,
                               'min': 3.4657359027997265,
                               'q': 1},
                'dropout': {'values': [0.3, 0.4, 0.5]},
                'epochs': {'value': 1},
                'fc_layer_size': {'values': [128, 256, 512]},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0},
                'optimizer': {'values': ['adam', 'sgd']}}}


## Step 2️⃣. Initialize the Sweep

Once you've defined the search strategy, it's time to set up something to implement it.

The clockwork taskmaster in charge of our Sweep is known as the _Sweep Controller_.
As each run completes, it will issue a new set of instructions
describing a new run to execute.
These instructions are picked up by _agents_
who actually perform the runs.

In a typical Sweep, the Controller lives on _our_ machine,
while the agents who complete runs live on _your_ machine(s),
like in the diagram below.
This division of labor makes it super easy to scale up Sweeps
by just adding more machines to run agents!

In [10]:
sweep_id = wandb.sweep(sweep_config, project="pytorch-sweeps-demo")

Create sweep with ID: nsbx9kob
Sweep URL: https://wandb.ai/swathi-anandram/pytorch-sweeps-demo/sweeps/nsbx9kob


## Step 3️. Run the Sweep agent

### Define Your Training Procedure

In [11]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from torchvision import datasets, transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        loader = build_dataset(config.batch_size)
        network = build_network(config.fc_layer_size, config.dropout)
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)

        for epoch in range(config.epochs):
            avg_loss = train_epoch(network, loader, optimizer)
            wandb.log({"loss": avg_loss, "epoch": epoch})           

This cell defines the four pieces of our training procedure:
`build_dataset`, `build_network`, `build_optimizer`, and `train_epoch`.

All of these are a standard part of a basic PyTorch pipeline,
and their implementation is unaffected by the use of W&B,
so we won't comment on them.

In [12]:
def build_dataset(batch_size):
   
    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.1307,), (0.3081,))])
    # download MNIST training dataset
    dataset = datasets.MNIST(".", train=True, download=True,
                             transform=transform)
    sub_dataset = torch.utils.data.Subset(
        dataset, indices=range(0, len(dataset), 5))
    loader = torch.utils.data.DataLoader(sub_dataset, batch_size=batch_size)

    return loader


def build_network(fc_layer_size, dropout):
    network = nn.Sequential(  # fully-connected, single hidden layer
        nn.Flatten(),
        nn.Linear(784, fc_layer_size), nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(fc_layer_size, 10),
        nn.LogSoftmax(dim=1))

    return network.to(device)
        

def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer


def train_epoch(network, loader, optimizer):
    cumu_loss = 0
    for _, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        # ➡ Forward pass
        loss = F.nll_loss(network(data), target)
        cumu_loss += loss.item()

        # ⬅ Backward pass + weight update
        loss.backward()
        optimizer.step()

        wandb.log({"batch loss": loss.item()})

    return cumu_loss / len(loader)

The cell below will launch an `agent` that runs `train` 5 times,
usingly the randomly-generated hyperparameter values returned by the Sweep Controller. Execution takes under 5 minutes.

In [13]:
wandb.agent(sweep_id, train, count=5)

wandb: Agent Starting Run: qgpjs220 with config:
wandb: 	batch_size: 50
wandb: 	dropout: 0.3
wandb: 	epochs: 1
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 0.010168744664737508
wandb: 	optimizer: adam
wandb: Currently logged in as: swathi-anandram (use `wandb login --relogin` to force relogin)


batch loss,█▄▄▄▃▂▃▂▃▃▂▃▂▁▂▂▂▃▂▃▄▃▂▃▁▃▂▂▂▂▃▃▂▂▂▂▃▂▁▃
epoch,▁
loss,▁
batch loss,0.75884
epoch,0
loss,0.58866


wandb: Agent Starting Run: eqh7tlgo with config:
wandb: 	batch_size: 121
wandb: 	dropout: 0.3
wandb: 	epochs: 1
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 0.019457808977274307
wandb: 	optimizer: sgd


batch loss,██▇▆▄▄▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▂▂▁▁▁▂▂▁▂▁▂▁▁▃
epoch,▁
loss,▁
batch loss,0.78433
epoch,0
loss,0.60527


wandb: Agent Starting Run: g1yvjhey with config:
wandb: 	batch_size: 33
wandb: 	dropout: 0.4
wandb: 	epochs: 1
wandb: 	fc_layer_size: 128
wandb: 	learning_rate: 0.08987058587313616
wandb: 	optimizer: sgd


batch loss,▅▂▁▃▃▄▅▃▄▂▆▄▆▄▄█▆▄▃▃█▄▅▄▅▄▃▄▅▅▂▄▄▄▄▄▃▃▂▃
epoch,▁
loss,▁
batch loss,3.95347
epoch,0
loss,1.68864


wandb: Agent Starting Run: u07srhds with config:
wandb: 	batch_size: 173
wandb: 	dropout: 0.3
wandb: 	epochs: 1
wandb: 	fc_layer_size: 512
wandb: 	learning_rate: 0.09446659940509344
wandb: 	optimizer: adam


batch loss,▁▅█▅▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁
loss,▁
batch loss,1.49296
epoch,0
loss,20.5337


wandb: Agent Starting Run: q978i15s with config:
wandb: 	batch_size: 36
wandb: 	dropout: 0.5
wandb: 	epochs: 1
wandb: 	fc_layer_size: 128
wandb: 	learning_rate: 0.0438388056384065
wandb: 	optimizer: adam


batch loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁
loss,▁
batch loss,1.79127
epoch,0
loss,2.1977
